##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Treine uma Rede Q Profunda com o TF-Agents

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/1_dqn_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/1_dqn_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução


Este exemplo mostra como treinar um agente [DQN (Redes Q Profundas)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) no ambiente Cartpole usando a biblioteca TF-Agents.

![Ambiente Cartpole](https://raw.githubusercontent.com/tensorflow/agents/master/docs/tutorials/images/cartpole.png)

Ele guiará você por todos os componentes de um pipeline de Aprendizado por Reforço (RL) para treinamento, avaliação e coleta de dados.

Para executar este código em tempo real, clique no link "Executar no Google Colab" acima.


## Configuração

Se você ainda não instalou as seguintes dependências, execute:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

In [ ]:
# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
tf.version.VERSION

## Hiperparâmetros

In [ ]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Ambiente

No Aprendizado por Reforço (RL), um ambiente representa a tarefa ou o problema a ser resolvido. Os ambientes padrão podem ser criados no TF-Agentes usando suítes de `tf_agents.environments`. O TF-Agents tem suítes para carregar ambientes de origens como OpenAI Gym, Atari e DM Control.

Carregue o ambiente CartPole a partir da suíte do OpenAI Gym. 

In [ ]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)

Você pode renderizar esse ambiente para ver como ele fica. Um pêndulo que balança livremente é acoplado a um carrinho. O objetivo é mover o carrinho para a direita ou para a esquerda a fim de manter o pêndulo em pé.

In [ ]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

O método `environment.step` pega uma `action` no ambiente e retorna uma tupla `TimeStep` com a próxima observação do ambiente e a recompensa para a ação.

O método `time_step_spec()` retorna a especificação para a tupla `TimeStep`. O atributo `observation` mostra o formato das observações, os tipos de dados e os intervalos dos valores permitidos. O atributo `reward` mostra os mesmos detalhes para a recompensa.


In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)

In [ ]:
print('Reward Spec:')
print(env.time_step_spec().reward)

O método `action_spec()` retorna o formato, os tipos de dados e os valores permitidos para ações válidas.

In [ ]:
print('Action Spec:')
print(env.action_spec())

No ambiente Cartpole:

- `observation` é um array de 4 floats:
    - a posição e a velocidade do carrinho
    -  a posição angular e a velocidade do pêndulo
- `reward` é um valor escalar float
- `action` é um número inteiro escalar que só aceita dois valores:
    - `0` — "mover para a esquerda"
    - `1` — "mover para a direita"


In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

Geralmente, dois ambientes são instanciados: um para treinamento e outro para avaliação. 

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

O ambiente Cartpole, como a maioria dos ambientes, é escrito em Python puro. Ele é convertido para o TensorFlow usando o wrapper `TFPyEnvironment`.

A API do ambiente original usa arrays do Numpy. O `TFPyEnvironment` os converte em `Tensors` para a compatibilidade com agentes e políticas do TensorFlow.


In [ ]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agente

O algoritmo usado para resolver um problema de RL é representado por um `Agent`. O TF-Agents oferece implementações padrão de uma variedade de `Agents`, incluindo:

- [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) (usado neste tutorial)
- [REINFORCE](https://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
- [DDPG](https://arxiv.org/pdf/1509.02971.pdf)
- [TD3](https://arxiv.org/pdf/1802.09477.pdf)
- [PPO](https://arxiv.org/abs/1707.06347)
- [SAC](https://arxiv.org/abs/1801.01290)

O agente DQN pode ser usado em qualquer ambiente com um espaço de ação discreto.

No coração de um Agente DQN está um `QNetwork`, um modelo de rede neural que pode aprender a prever `QValues` (retornos esperados) para todas as ações, a partir de uma observação do ambiente.

Vamos usar `tf_agents.networks.` para criar uma `QNetwork`. A rede consistirá em uma sequência de camadas `tf.keras.layers.Dense`, em que a camada final terá 1 saída para cada ação possível.

In [ ]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

Agora use o `tf_agents.agents.dqn.dqn_agent` para instanciar um `DqnAgent`. Além de `time_step_spec`, `action_spec` e QNetwork, o construtor do agente também exige um otimizador (nesse caso, `AdamOptimizer`), uma função de perda e um contador de passos de número inteiro.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## Políticas

Uma política define a maneira que um agente age em um ambiente. Geralmente, a meta do aprendizado por reforço é treinar o modelo subjacente até que a política produza o resultado desejado.

Neste tutorial:

- O resultado desejado é manter o pêndulo em pé no carrinho.
- A política retorna uma ação (esquerda ou direita) para cada observação `time_step`.

Os agentes contêm duas políticas:

- `agent.policy` — a principal política que é usada para avaliação e implantação.
- `agent.collect_policy` — uma segunda política que é usada para coleta de dados.


In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

As políticas podem ser criadas independentemente dos agentes. Por exemplo, use `tf_agents.policies.random_tf_policy` para criar uma política que seleciona aleatoriamente uma ação para cada `time_step`.

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

Para obter uma ação de uma política, chame o método `policy.action(time_step)`. O `time_step` contém a observação do ambiente. Esse método retorna um `PolicyStep`, que é uma tupla nomeada com três componentes:

- `action` — a ação a ser realizada (nesse caso, `0` ou `1`)
- `state` — usado para políticas stateful (ou seja, baseadas em RNN)
- `info` — dados auxiliares, como probabilidades log de ações

In [ ]:
example_environment = tf_py_environment.TFPyEnvironment(
    suite_gym.load('CartPole-v0'))

In [ ]:
time_step = example_environment.reset()

In [ ]:
random_policy.action(time_step)

## Métricas e avaliação

A métrica mais comum usada para avaliar uma política é o retorno médio. O retorno é a soma das recompensas obtidas ao executar uma política em um ambiente para um episódio. Vários episódios são executados, criando um retorno médio.

A seguinte função computa o retorno médio de uma política, considerando a política, o ambiente e o número de episódios.


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

A execução dessa computação em `random_policy` mostra um desempenho de linha de base no ambiente.

In [ ]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)

## Buffer de replay

Para rastrear os dados coletados do ambiente, vamos usar o [Reverb](https://deepmind.com/research/open-source/Reverb), um sistema de replay eficiente, extensível e fácil de usar do Deepmind. Ele armazena dados de experiências quando coletamos trajetórias e é consumido durante o treinamento.

Esse buffer de replay é construído usando as especificações que descrevem os tensores a serem armazenados, que podem ser obtidas do agente usando agent.collect_data_spec.


In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

Para a maioria dos agentes, `collect_data_spec` é uma tupla nomeada `Trajectory`, que contém as especificações para observações, ações, recompensas e outros itens.

In [ ]:
agent.collect_data_spec

In [ ]:
agent.collect_data_spec._fields

## Coleta de dados

Agora execute a política aleatória no ambiente para alguns passos, registrando os dados no buffer de replay.

Usamos aqui o 'PyDriver' para executar o loop de coleta de experiência. Saiba mais sobre o driver TF Agents no nosso [tutorial de drivers](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
#@test {"skip": true}
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_py_env.reset())

O buffer de replay é agora uma coleção de Trajetórias.

In [ ]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

O agente precisa de acesso ao buffer de replay. Isso é fornecido ao criar um pipeline `tf.data.Dataset` iterável que alimentará dados ao agente.

Cada linha do buffer de replay só armazena um único passo de observação. Porém, como o Agente DQN precisa tanto da observação atual quanto da próxima para computar a perda, o pipeline do dataset usará duas linhas adjacentes como amostra para cada item no lote (`num_steps=2`).

Esse dataset também é otimizado executando células paralelas e fazendo a pré-busca dos dados.

In [ ]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

In [ ]:
iterator = iter(dataset)
print(iterator)

In [ ]:
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data 
# to the collection of individual trajectories shown earlier.

# iterator.next()

## Treinando o agente

Duas coisas precisam acontecer durante o loop de treinamento:

- coletar dados do ambiente
- usar esses dados para treinar as redes neurais do agente

Esse exemplo também avalia regularmente a política e imprime a pontuação atual.

A execução do código a seguir levará cerca de 5 minutos.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## Visualização


### Plots

Use `matplotlib.pyplot` para traçar a melhoria da política durante o treinamento.

Uma iteração de `Cartpole-v0` consiste em 200 timesteps. O ambiente dá uma recompensa de `+1` para cada passo que o pêndulo permanece em pé, então o retorno máximo para um episódio é 200. O gráfico mostra o retorno aumentando em direção a esse máximo a cada vez que é avaliado durante o treinamento. (Pode ser um pouco instável e não aumentar de maneira monótona toda vez).

In [ ]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### Vídeos

Os gráficos são ótimos. Porém, é muito mais incrível ver um agente realmente realizando uma tarefa em um ambiente.

Primeiro, crie uma função para incorporar vídeos no notebook.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Agora, itere alguns episódios do jogo Cartpole com o agente. O ambiente Python subjacente ("dentro" do wrapper de ambiente do TensorFlow) oferece um método `render()`, que gera uma imagem do estado do ambiente. Isso pode ser coletado em um vídeo.

In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")

Por diversão, compare o agente treinado (acima) a um agente que se move aleatoriamente. (Ele não tem um desempenho tão bom.)

In [ ]:
create_policy_eval_video(random_policy, "random-agent")